In [1]:
import pandas as pd
import numpy as np
import requests
import os
import shutil
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
def mvp():
    mvps = pd.read_csv('data/mvp.csv')
    return mvps[['Player', 'Pts Won', 'Pts Max', 'Share', 'Year']]

In [4]:
def players():
    players = pd.read_csv("data/players.csv")
    players = players.drop(columns = ['Unnamed: 0', 'Rk'])
    players['Player'] = players['Player'].str.replace('*','')
    if players.shape[0]==1:
        return players
    else:
        players = players[players['Tm'] == 'TOT']
        players['Tm'] = players.iloc[-1,:]['Tm']
    players = players.groupby(['Player', 'Year']).apply(players)
    players.index = players.index.droplevel()
    players.index = players.index.droplevel()
    df_combinado = players.merge(mvps, how = 'outer', on = ['Player', 'Year'])
    df_combinado[['Pts Won', 'Pts Max', 'Share']] = df_combinado[['Pts Won', 'Pts Max', 'Share']].fillna(0)
    df_combinado = df_combinado.apply(pd.to_numeric, errors = 'ignore')
    return df_combinado

In [12]:
def nicknames():
    nicknames = pd.read_csv('data/nicknames.txt')
    nicknames = {}
    with open("data/nicknames.txt") as f:
        lines = f.readlines()
        for line in lines[1:]:
            abreviado,name = line.replace("\n","").split(",")
            nicknames[abreviado] = name
    df_combinado["Team"] = df_combinado["Tm"].map(nicknames)
    return df_combinado


In [14]:
def teams():
    teams = pd.read_csv('data/teams.csv')
    teams = teams[~teams['W'].str.contains('Division')]
    teams['Team'] = teams['Team'].str.replace('*', '')
    teams['GB'] = teams['GB'].replace('—', '0')
    teams = teams.drop(columns = ['Unnamed: 0'])
    return teams

In [23]:
def final_stats():
    df_final = df_combinado.merge(teams, how = 'outer', on = ['Team','Year'])
    df_final = df_final.apply(pd.to_numeric, errors = 'ignore')
    return df_final